# AutoDiff by Symboic Representation in Julia

In [1]:
using Symbolics

In [2]:
i(x) = x
f(x) = 3x^2
g(x) = 2x^2
h(x) = x^2
w_vec = [i, h, g, f]

@variables x 

1-element Vector{Num}:
 x

In [3]:
function forward_fn(w_vec, x, i::Int)
    y = w_vec[i](x)
    i == size(w_vec)[1] ? y : [y; forward_fn(w_vec,y,i+1)] 
end

forward_fn (generic function with 1 method)

In [4]:
x_vec = forward_fn(w_vec, x, 1)
display(x_vec)

4-element Vector{Num}:
             x
           x^2
        2(x^4)
 (12//1)*(x^8)

In [14]:
function gradient(w_i, x_i_1, x)
    dy = expand_derivatives(Differential(x)(w_i(x)))
    substitute(dy, (Dict(x=>x_i_1,)))
end

function reverse_autodiff(w_vec, x_vec, x, i::Int)
    if i == 1
        1
    else
        gradient(w_vec[i],x_vec[i-1],x) * 
            reverse_autodiff(w_vec, x_vec, x, i-1)
    end
end

reverse_autodiff (generic function with 1 method)

In [15]:
y_sad = x_vec[end]
display(y_sad)
dy_sad = reverse_autodiff(w_vec, x_vec, x, size(w_vec)[1])
display(dy_sad)

(12//1)*(x^8)

96(x^7)

## Check by theory

In [11]:
y_th = f(g(h(x)))
display(y_th)
dy_th = expand_derivatives(Differential(x)(y_th))
display(dy_th)

(12//1)*(x^8)

(96//1)*(x^7)